In [48]:
import requests as rs
from zipfile import ZipFile
from io import BytesIO
import pandas as pd
from azure.storage.blob import ContainerClient,BlobServiceClient
from io import StringIO
import numpy as np
import json
import datetime as dt
import warnings


In [49]:
pd.set_option('max_colwidth', None)

In [50]:

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

pd.set_option('display.max_rows', None)


In [51]:

def getAccessToken(request_body):
    req = rs.post(
                  "https://login.windows.net/6e75cca6-47f0-47a3-a928-9d5315750bd9/oauth2/token"
                  ,data = request_body)
    access_token = json.loads(req.text)['access_token']
    return access_token

def getInvoices (base_url):
    relative_url = '/v1/invoices'

    response = json.loads(
    rs.get(
            f"{base_url}{relative_url}",
           headers=headers)
    .content)

    main_df=pd.DataFrame() # Main DataFrame, currently empty

    columns = list(response['items'][0].keys())

    for row in response['items']:
        row_df = pd.DataFrame.from_dict([row]) #Each row gets converted to a DataFrame
        main_df = pd.concat([main_df,row_df],ignore_index=True) #Then gets appended to the Main DataFrame
        
    return main_df

def getInvoiceLineItems(base_url, invoiceId):
    relative_url = f'/v1/invoices/{invoiceId}/lineitems/OneTime/BillingLineItems?size=5000'

    response = json.loads(
        rs.get(
                f"{base_url}{relative_url}",
               headers=headers)
        .content)

    main_df=pd.DataFrame()

    columns = list(response['items'][0].keys())

    for row in response['items']:
        sub_df = pd.DataFrame.from_dict([row])
        main_df = pd.concat([main_df,sub_df],ignore_index=True)
    return main_df


In [52]:

secrets = json.load(open(file='./secrets.json'))

refresh_token = secrets['refresh_token']

app_id = secrets['app_id']

app_secret = secrets['app_secret']

request_body = {
        "grant_type": "refresh_token",
        "refresh_token": refresh_token,
        "scope": "openid",
        "resource": "https://api.partnercenter.microsoft.com",
        "client_id": app_id,
        "client_secret": app_secret,
    }

base_url = (
'https://api.partnercenter.microsoft.com'
)

isAccessTokenObtained = 0


In [53]:

try:
    access_token = getAccessToken(request_body = request_body)
    print('Refresh token valid, Access token obtained.')
    isAccessTokenObtained = 1
except:
    print('Refresh token expired.')




Refresh token valid, Access token obtained.


In [54]:




if isAccessTokenObtained == 1:
    headers = {'Authorization': 'Bearer ' + access_token}

    print('getting invoices...')
    invoices_df = getInvoices(base_url = base_url)


getting invoices...


In [55]:
invoices_df.head(2)

,id,invoiceDate,billingPeriodStartDate,billingPeriodEndDate,totalCharges,paidAmount,currencyCode,currencySymbol,pdfDownloadLink,invoiceDetails,documentType,state,invoiceType,links,attributes
0,D080003NJ7,2023-11-14T00:00:00Z,2023-10-13T00:00:00Z,2023-11-12T00:00:00Z,233025.31,-233025.31,INR,₹,/invoices/D080003NJ7/documents/statement,"[{'invoiceLineItemType': 'billing_line_items', 'billingProvider': 'office', 'links': {'self': {'uri': '/invoices/Recurring-D080003NJ7/lineitems/Office/BillingLineItems', 'method': 'GET', 'headers': []}}, 'attributes': {'objectType': 'InvoiceDetail'}}]",invoice,,Recurring,"{'self': {'uri': '/invoices/Recurring-D080003NJ7', 'method': 'GET', 'headers': []}}",{'objectType': 'Invoice'}
1,D080003N0B,2023-10-14T00:00:00Z,2023-09-13T00:00:00Z,2023-10-12T00:00:00Z,287895.74,-287895.74,INR,₹,/invoices/D080003N0B/documents/statement,"[{'invoiceLineItemType': 'billing_line_items', 'billingProvider': 'office', 'links': {'self': {'uri': '/invoices/Recurring-D080003N0B/lineitems/Office/BillingLineItems', 'method': 'GET', 'headers': []}}, 'attributes': {'objectType': 'InvoiceDetail'}}]",invoice,,Recurring,"{'self': {'uri': '/invoices/Recurring-D080003N0B', 'method': 'GET', 'headers': []}}",{'objectType': 'Invoice'}


In [57]:
relative_url = f'/v1/invoices/Recurring-D080003NJ7/lineitems/Office/BillingLineItems'

response = rs.get(
        f"{base_url}{relative_url}",
        headers=headers).content

response = json.loads(response)
response['items'][1]

{'partnerId': '6e75cca6-47f0-47a3-a928-9d5315750bd9',
 'customerId': 'c11d49c2-6688-4d98-830d-84671a8419a1',
 'customerName': 'TechN Vision Ventures Limited',
 'mpnId': 2240443,
 'tier2MpnId': -1,
 'orderId': '569423907152295365',
 'invoiceNumber': 'D080003NJ7',
 'subscriptionId': 'j0cd0wEAAAAAAAEA',
 'syndicationPartnerSubscriptionNumber': '86ebb7e6-e654-4c97-81a9-ee2259ecfc55',
 'offerId': 'dffa072b-d246-4f59-a99d-6d588e839629',
 'durableOfferId': '2a727ae4-f201-497d-a9d6-c6a892df4a87',
 'offerName': 'Microsoft 365 Business Basic India',
 'domainName': 'recruitsharp.com',
 'billingCycleType': 'Monthly',
 'subscriptionName': 'Office 365 Business Essentials India',
 'subscriptionDescription': 'Microsoft 365 Business Basic India',
 'subscriptionStartDate': '2023-01-16T00:00:00Z',
 'subscriptionEndDate': '2024-01-16T00:00:00Z',
 'chargeStartDate': '2023-10-16T00:00:00Z',
 'chargeEndDate': '2023-11-15T00:00:00Z',
 'chargeType': 'Cycle fee',
 'unitPrice': 100.0,
 'quantity': 8,
 'amount': 

In [12]:
response

{'totalCount': 145,
 'items': [{'id': 'D080003NJ7',
   'invoiceDate': '2023-11-14T00:00:00Z',
   'billingPeriodStartDate': '2023-10-13T00:00:00Z',
   'billingPeriodEndDate': '2023-11-12T00:00:00Z',
   'totalCharges': 233025.31,
   'paidAmount': -233025.31,
   'currencyCode': 'INR',
   'currencySymbol': '₹',
   'pdfDownloadLink': '/invoices/D080003NJ7/documents/statement',
   'invoiceDetails': [{'invoiceLineItemType': 'billing_line_items',
     'billingProvider': 'office',
     'links': {'self': {'uri': '/invoices/Recurring-D080003NJ7/lineitems/Office/BillingLineItems',
       'method': 'GET',
       'headers': []}},
     'attributes': {'objectType': 'InvoiceDetail'}}],
   'documentType': 'invoice',
   'state': '',
   'invoiceType': 'Recurring',
   'links': {'self': {'uri': '/invoices/Recurring-D080003NJ7',
     'method': 'GET',
     'headers': []}},
   'attributes': {'objectType': 'Invoice'}},
  {'id': 'D080003N0B',
   'invoiceDate': '2023-10-14T00:00:00Z',
   'billingPeriodStartDate':

In [ ]:

    print('Filtering invoices to One-Time...')
    invoices_df = invoices_df[invoices_df['invoiceType']=='OneTime']

    invoiceIdList = list(invoices_df['id'])
    invoiceDateList = list(invoices_df['invoiceDate'])

    zipp = zip(invoiceIdList,invoiceDateList)


In [8]:
invoice_df = getInvoiceLineItems(base_url = base_url, invoiceId = 'D080003NJ7')


KeyError: 'items'

In [ ]:

    print(invoice_df[
        (invoice_df['customerName'].str.startswith('TATA'))
            ][
        ['productName','skuName','termAndBillingCycle','totalForCustomer','chargeStartDate','chargeEndDate']
            ].sort_values('productName'))




